In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(13060, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(13060, 19)
Patient Number 	 737
State Patient Number 	 10960
Date Announced 	 738
Age Bracket 	 11552
Gender 	 10674
Detected City 	 11600
Detected District 	 2909
Detected State 	 739
State code 	 740
Current Status 	 740
Notes 	 1929
Contracted from which Patient (Suspected) 	 11633
Nationality 	 11762
Type of transmission 	 10118
Status Change Date 	 850
Source_1 	 929
Source_2 	 9933
Source_3 	 12736
Backup Notes 	 12700


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(13060, 19)


Patient Number                                 737
State Patient Number                         10960
Date Announced                                 738
Age Bracket                                  11552
Gender                                       10674
Detected City                                11600
Detected District                             2909
Detected State                                 739
State code                                     740
Current Status                                 740
Notes                                         1929
Contracted from which Patient (Suspected)    11633
Nationality                                  11762
Type of transmission                         10118
Status Change Date                             850
Source_1                                       929
Source_2                                      9933
Source_3                                     12736
Backup Notes                                 12700
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(12321, 19)


Patient Number                                   0
State Patient Number                         10221
Date Announced                                   0
Age Bracket                                  10814
Gender                                        9936
Detected City                                10861
Detected District                             2171
Detected State                                   0
State code                                       1
Current Status                                   2
Notes                                         1191
Contracted from which Patient (Suspected)    10894
Nationality                                  11024
Type of transmission                          9380
Status Change Date                             112
Source_1                                       191
Source_2                                      9195
Source_3                                     11998
Backup Notes                                 11961
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
8937,8938,AP-P418,12/04/2020,55,M,Pedda,Kurnool,Andhra Pradesh,AP,Hospitalized,Details awaited,NaN,NaN,NaN,12/04/2020,https://twitter.com/ArogyaAndhra/status/124933...,NaN,NaN,NaN
8365,8366,NaN,11/04/2020,NaN,NaN,NaN,Bhopal,Madhya Pradesh,MP,Hospitalized,Details awaited,NaN,NaN,NaN,11/04/2020,https://twitter.com/JansamparkMP/status/124900...,NaN,NaN,NaN
7264,7265,NaN,10/04/2020,NaN,NaN,NaN,NaN,Delhi,DL,Hospitalized,Details awaited,NaN,NaN,NaN,10/04/2020,https://twitter.com/CMODelhi/status/1248629443...,NaN,NaN,NaN
3077,3078,NaN,03/04/2020,NaN,NaN,NaN,Jaipur,Rajasthan,RJ,Hospitalized,NaN,NaN,NaN,NaN,03/04/2020,http://www.rajswasthya.nic.in/,NaN,NaN,NaN
6851,6852,NaN,10/04/2020,NaN,NaN,NaN,Evacuees,Rajasthan,RJ,Hospitalized,Details awaited,NaN,NaN,NaN,10/04/2020,https://twitter.com/PIBJaipur/status/124853725...,NaN,NaN,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
7182,7183,NaN,10/04/2020,NaN,NaN,NaN,NaN,Delhi,DL,Hospitalized,Details awaited,NaN,NaN,NaN,10/04/2020,https://twitter.com/CMODelhi/status/1248629443...,NaN,NaN,NaN
3606,3607,NaN,04/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,NaN,NaN,NaN,NaN,NaN,https://twitter.com/rajeshtope11/status/124648...,NaN,NaN,NaN
4557,4558,NaN,06/04/2020,NaN,NaN,NaN,S.A.S. Nagar,Punjab,PB,Hospitalized,Details awaited,NaN,NaN,NaN,06/04/2020,"State Bulletin, 6th Apr",NaN,NaN,NaN
5491,5492,NaN,08/04/2020,NaN,NaN,NaN,Srinagar,Jammu and Kashmir,JK,Hospitalized,Details awaited,NaN,NaN,NaN,08/04/2020,https://twitter.com/kansalrohit69/status/12478...,https://twitter.com/diprjk/status/124785438455...,NaN,NaN
7039,7040,NaN,10/04/2020,NaN,NaN,NaN,Bhopal,Madhya Pradesh,MP,Hospitalized,Details awaited,NaN,NaN,NaN,10/04/2020,https://twitter.com/JansamparkMP/status/124861...,NaN,NaN,NaN


In [19]:
p_df.to_csv('patients_data.csv', index=False)